In [ ]:
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.signal import convolve2d
from tqdm import tqdm

#### Get an estimate for f using <br><br> (i) Wiener filter <br> (ii) using MAP estimation <br> (iii) any deep network based approach under the following cases.

In [ ]:
img = Image.open("cute_pup.jpg")
img = img.resize((100,100))
img = np.array(img)
img = 0.3 * img[:,:,0] + 0.6 * img[:,:,1] + 0.1 * img[:,:,2]
plt.imshow(img,cmap='gray')
plt.title("orginal image")
plt.show()

### case 1

In [ ]:
A = np.identity(100)
n = np.clip(np.random.normal(50,30,size=img.shape),0,255)
g_img = (A @ img) + n 
plt.imshow(g_img,cmap='gray')
cv2.imwrite("noise.jpg",g_img)

### Wiener's Filter

In [ ]:
H = np.fft.fft2((1/500) * A,s=img.shape)
K = 0.7 * np.ones(H.shape)
H_1 = np.conj(H)
Wiener_filter = H_1/(np.abs(H)**2 + K)
G = np.fft.fft2(g_img)
filtered_img = G * Wiener_filter
filtered_img = np.fft.ifft2(filtered_img)
plt.imshow(np.abs(filtered_img),cmap='gray')
plt.title("Restoration using Wiener's Filter")
plt.show()

### MAP Estimation

In [ ]:
def estimate_psf(degraded_image, iterations):
    # Initialize PSF as a delta function (impulse)
    psf = np.zeros_like(degraded_image)
    psf[degraded_image.shape[0] // 2, degraded_image.shape[1] // 2] = 1.0
    
    # Perform Richardson-Lucy deconvolution iterations
    for i in range(iterations):
        # Estimate the restored image using the current PSF
        restored_image = deconvolve(degraded_image, psf)
        
        # Estimate the error image (residual)
        error_image = degraded_image / np.maximum(restored_image, 1e-5)
        
        # Update PSF using the error image
        psf = update_psf(psf, error_image)
        
        # Normalize PSF to sum to 1
        psf /= np.sum(psf)
        
    return psf

def deconvolve(image, psf,epsilon=1e-10):
    # Perform Richardson-Lucy deconvolution
    restored_image = np.real(np.fft.ifft2(np.fft.fft2(image) / np.maximum(np.fft.fft2(psf),epsilon)))
    return restored_image

def update_psf(psf, error_image):
    # Update PSF using error image (e.g., by convolving error image with the transpose of the image)
    updated_psf = np.real(np.fft.ifft2(np.fft.fft2(error_image) * np.conj(np.fft.fft2(psf))))
    return updated_psf


In [7]:
epochs = 2

new_out = g_img.copy()
FIL = A.copy()

for i in tqdm(range(epochs)):
    new_out = convolve2d(new_out, FIL, mode='same', boundary='wrap')
    new_out_psf = estimate_psf(new_out,10)
    prob = multivariate_normal(mean = (new_out * new_out_psf).flatten(),cov = (30*A).flatten()).pdf(new_out)
    new_out = prob / np.sum(prob)
    new_out = 1 - new_out

plt.imshow(new_out)

### Deep Learning model

In [ ]:
import replicate


input = {
    "image" : "https://github.com/vrs-darkness/Assignment/blob/main/IVP%20LAB/noise.jpg"
}
api = replicate.Client(api_token="r8_HVMiznvj0NEuYa0U4cxBPQoPnhbJJr24Hh5Xh")
output = api.run(
    "cszn/scunet:df9a3c1dbc6c1f7f4c2d244f68dffa2699a169cf5e701e0d6a009bf6ff507f26",
    input = input
)
print(output)

### case 2

In [ ]:
h = np.identity(20)
n = np.clip(np.random.normal(10,30,size=img.shape),0,255)
g_img = np.add(cv2.filter2D(src=img,ddepth = -1,kernel = h),n)

In [ ]:
plt.imshow(g_img,cmap='gray')
plt.title("Noise Image")
plt.axis('off')
cv2.imwrite("shake-pup.jpg",g_img)
plt.show()

### Wiener's Filter

In [ ]:
H = np.fft.fft2(h,s=img.shape)
K = 10
H_1 = np.conj(H)
Wiener_filter = H_1/(H * H_1 + K)
G = np.fft.fft2(g_img)
filtered_img = Wiener_filter * G
filtered_img = np.fft.ifft2(filtered_img)

In [ ]:
plt.imshow(np.abs(filtered_img),cmap='gray')
plt.title("Restoration using Wiener's Filter")
plt.show()

### Map Estimation

In [ ]:
epochs = 2

new_out = g_img.copy()
FIL = h.copy()

for i in range(epochs):
    new_out = convolve2d(new_out, FIL, mode='same', boundary='wrap')
    new_out_psf = estimate_psf(new_out,10)
    prob = multivariate_normal(mean = (new_out * new_out_psf).flatten(),cov = (30*A).flatten()).pdf(new_out)
    new_out = prob / np.sum(prob)
    new_out = 1 - new_out

plt.imshow(new_out)

### Deep Learning meathod

In [ ]:
import replicate


input = {
    "image" : "https://github.com/vrs-darkness/Assignment/blob/main/IVP%20LAB/shake-pup.jpg",
    "task_type" : "Image Denoising"
}
api = replicate.Client(api_token="r8_HVMiznvj0NEuYa0U4cxBPQoPnhbJJr24Hh5Xh")
output = api.run(
    "megvii-research/nafnet:e116b6df8437d9c562f9de2a86cea6fd76a96705e502f091457926bbe989436c",
    input = input
)
print(output)


### END